# ０．事前準備

## ０ー１．データセットのダウンロード

In [ ]:
# https://qiita.com/namakemono/items/c963e75e0af3f7eed732
!curl -sc /tmp/cookie "https://drive.google.com/uc?export=download&id=10V_P59RLK7w_6buGyi-ArILZMNTPCy8Q" > /dev/null
!CODE="$(awk '/_warning_/ {print $NF}' /tmp/cookie)"
!curl -Lb /tmp/cookie "https://drive.google.com/uc?export=download&confirm=${CODE}&id=10V_P59RLK7w_6buGyi-ArILZMNTPCy8Q" -o archive.zip
!unzip -q archive.zip

## ０－２．学習済みモデルをアップロード

In [ ]:
# https://drive.google.com/file/d/1JXPHg4brau1T93z79VNr4VqLeCEx2CcW/view?usp=share_link 
# からモデルファイルを一旦自身のPCにダウンロードいただき、
# そのうえで左のエクスプローラーにアップロードしてください。



---



# １．改めてPyTorchで学習済みモデルを推論

## １－１．まずはColabの下で動いているCPUを確認
Intel Xeonが動いているか確認してください。他メーカーのCPUの場合はこの先のプログラムが正常に動作しない可能性があります。

In [ ]:
!lscpu

## １－２．必要なライブラリーをインポート

In [ ]:
import numpy as np 
import pandas as pd

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torchvision import transforms as T
import torchvision
import torch.nn.functional as F
from torch.autograd import Variable

import statistics

from PIL import Image
import cv2
import albumentations as A

import time
import os
from tqdm.notebook import tqdm

!pip install -q segmentation-models-pytorch
!pip install -q torchsummary

from torchsummary import summary
import segmentation_models_pytorch as smp

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## １－３．パスの設定

In [ ]:
IMAGE_PATH = 'Fuehuki_Dataset/JPEGImages/'
MASK_PATH = 'Fuehuki_Dataset/SegmentationClass/'
MODEL_PATH = 'drone_Trained_Fuefuki.pth'

## １－４．必要な関数を定義

In [ ]:
mapping = {(0, 0, 0): 0,
           (150, 143, 9): 1,
           (192, 128, 32): 1,
           (150, 143, 3): 1,
            }    

def inverse_dict(map):
    return {v:k for k, v in map.items() }

reverse_mapping = inverse_dict(mapping)

def visualize(temp):
    r = temp.copy()
    g = temp.copy()
    b = temp.copy()
    for l in range(0,len(reverse_mapping)):
        r[temp==l]=reverse_mapping[l][0]
        g[temp==l]=reverse_mapping[l][1]
        b[temp==l]=reverse_mapping[l][2]

    rgb = np.zeros((temp.shape[1], temp.shape[2],3))

    rgb[:,:,0] = (r)
    rgb[:,:,1] = (g)
    rgb[:,:,2] = (b)
    return rgb

def predict_image(model, image, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
    model.eval()
    t = T.Compose([T.ToTensor(), T.Normalize(mean, std)])
    image = t(image)
    model.to(device); image=image.to(device)
    with torch.no_grad():
        image = image.unsqueeze(0)
        
        start = time.time()
        output = model(image)
        proc_time = '{:.2f}'.format(time.time() - start)
        print(f'推論処理時間：{proc_time}秒')
        masked = torch.argmax(output, dim=1)
        masked = masked.cpu().squeeze(0)
    return masked

t_test = A.Resize(768, 1152, interpolation=cv2.INTER_NEAREST)

## １－５．学習済みモデルの読み込み

In [ ]:
model = smp.Unet('mobilenet_v2', encoder_weights='imagenet', classes=24, activation=None, encoder_depth=5, decoder_channels=[256, 128, 64, 32, 16])
model.load_state_dict(torch.load(MODEL_PATH))

## １－６．モデルの推論実行（PyTorch）
一枚の画像を推論するのにどのくらい時間がかかるか確認しましょう。

In [ ]:
filepath = IMAGE_PATH + 'DJI_0075.jpg'
img = cv2.imread(filepath)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

aug = t_test(image=img)
img = Image.fromarray(aug['image'])

pred_mask = predict_image(model, img)
pred_mask = pred_mask.cpu().numpy().copy()
pred_mask = pred_mask.reshape(1,pred_mask.shape[0],pred_mask.shape[1])

print(filepath)

%matplotlib inline
import matplotlib.pyplot as plt
plt.figure(figsize=(20,16))
plt.subplot(1,2,1)
plt.imshow(img)
plt.subplot(1,2,2)
plt.imshow(visualize(pred_mask))



---



# ２．ここからモデル最適化

## ２－１．インテル® OpenVINOツールキットのインストール

In [ ]:
!pip install openvino openvino-dev[extras] onnx

## ２－２．学習済みモデル（.pthファイル）をONNX形式に変換

In [ ]:
def Convert_ONNX(trained_model): 

    # set the model to inference mode 
    trained_model.eval() 

    # Let's create a dummy input tensor  
    dummy_input = torch.randn(1, 3, 768, 1152, requires_grad=True) 

    # Export the model   
    torch.onnx.export(trained_model,         # model being run 
         dummy_input,       # model input (or a tuple for multiple inputs) 
         "Drone_Segmentation_Custum.onnx",       # where to save the model  
         export_params=True,  # store the trained parameter weights inside the model file 
         opset_version=11,    # the ONNX version to export the model to 
         do_constant_folding=True,  # whether to execute constant folding for optimization 
         input_names = ['modelInput'],   # the model's input names 
         output_names = ['modelOutput'], # the model's output names 
         dynamic_axes={'modelInput' : {0 : 'batch_size'},    # variable length axes 
                                'modelOutput' : {0 : 'batch_size'}}) 
    print(" ") 
    print('Model has been converted to ONNX') 

Convert_ONNX(model)

## ２－３．OpenVINO Model Opptimizerを用いてモデルをCPUに最適化

In [ ]:
!mo --input_model Drone_Segmentation_Custum.onnx --input_shape [1,3,768,1152]

## ２－４．必要なライブラリーをインポート

In [ ]:
%matplotlib inline
import cv2
import matplotlib.pyplot as plt
import numpy as np
import sys
from openvino.runtime import Core

## ２－５．OpenVINOのエンジンを作成

In [ ]:
ie = Core()

model = ie.read_model(model="Drone_Segmentation_Custum.xml")
compiled_model = ie.compile_model(model=model, device_name="CPU")

input_layer_ir = compiled_model.input(0)
output_layer_ir = compiled_model.output(0)

## ２－６．OpenVINOのエンジン上で最適化モデルを実行
最適化後は推論にどのくらいかかるか確認しましょう。

In [ ]:
filepath = IMAGE_PATH + 'DJI_0075.jpg'
image = cv2.imread(filepath)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

aug = t_test(image=image)
image = Image.fromarray(aug['image'])
original_image = image

mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]
t = T.Compose([T.ToTensor(), T.Normalize(mean, std)])
image = t(image)
image = image.to('cpu').detach().numpy().copy()
input_image = np.expand_dims(
    image, 0
)

# Run the inference.
start = time.time()
output = compiled_model([input_image])[output_layer_ir]
proc_time = '{:.2f}'.format(time.time() - start)
print(f'推論処理時間：{proc_time}秒')

# Prepare data for visualization.
segmentation_mask = np.argmax(output, axis=1)

plt.figure(figsize=(20,16))
plt.subplot(1,2,1)
plt.imshow(original_image)
plt.subplot(1,2,2)
plt.imshow(visualize(segmentation_mask))



---



# ３．ここからモデル量子化

## ３－１．モデルの量子化を実行
最もシンプルな方法で量子化を実施します。設定がシンプルな分、量子化後の精度は保証されません。

まずは下記URLから「simplified_mode_template.json」をダウンロードし、本Colab上にアップロードください。
https://drive.google.com/file/d/1TXmrXOnkVcPppR3bQz6zcehl_PD_-pto/view?usp=share_link

Colab上で"simplified_mode_template.json"を開き、下記の項目をそれぞれ実際の値で上書き下さい。
* MODEL_PATH ⇒ .xml path
* PATH_TO_WEIGHTS ⇒ .bin path
* PATH_TO_SOURCE ⇒ 画像データフォルダーパス

最後に以下のコマンドを実行すると量子化が開始します。
（かなり時間が掛かることがあります）

In [ ]:
!pot -c simplified_mode_template.json

## ３－２．OpenVINOのエンジンを作成
results/model_name_DefaultQuantization/YYYY-MM-DD_HH-MM-SS/optimized/model_name.xml
というファイルができていると思うので、それを読み込む

In [ ]:
ie = Core()

model = ie.read_model(model="results/model_name_DefaultQuantization/2023-01-12_05-31-29/optimized/model_name.xml")
compiled_model = ie.compile_model(model=model, device_name="CPU")

input_layer_ir = compiled_model.input(0)
output_layer_ir = compiled_model.output(0)

## ３－３．OpenVINOのエンジン上で最適化モデルを実行

In [ ]:
filepath = IMAGE_PATH + 'DJI_0075.jpg'
image = cv2.imread(filepath)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

aug = t_test(image=image)
image = Image.fromarray(aug['image'])
original_image = image

mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]
t = T.Compose([T.ToTensor(), T.Normalize(mean, std)])
image = t(image)
image = image.to('cpu').detach().numpy().copy()
input_image = np.expand_dims(
    image, 0
)

# Run the inference.
start = time.time()
output = compiled_model([input_image])[output_layer_ir]
proc_time = '{:.2f}'.format(time.time() - start)
print(f'推論処理時間：{proc_time}秒')

# Prepare data for visualization.
segmentation_mask = np.argmax(output, axis=1)

plt.figure(figsize=(20,16))
plt.subplot(1,2,1)
plt.imshow(original_image)
plt.subplot(1,2,2)
plt.imshow(visualize(segmentation_mask))